In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# from pyathena import connect
# from pyathena.pandas.util import as_pandas
# import urllib.request as request

# cursor = connect(
#  region_name="eu-west-1",
#  s3_staging_dir="s3://wih.aws-athena-query-results.eu-west-1/",
#  work_group="primary"
#  ).cursor()

pd.set_option("display.max_columns", None)

# Offerta Luiss
- Contiene la lista di *professioni per corso di studi*.

- Il codice della professione fa riferimento alla **classificazione CP2021** dell'ISTAT.

- Abbiamo quindi bisogno di transcodificarlo nel **codice ISCO-08** presente nel database OJA

In [4]:
offerta = pd.read_excel("./data/offerta_luiss/offerta_formativa_luiss.xlsx")
offerta.columns = ['percorso', 'facolta', 'corso_studi_id', 'corso_studi_desc', 'cp2021', 'cp2021_desc']
print("Numero di righe del file:", offerta.shape[0])
print("cp2021 univoci:", offerta.cp2021.nunique())
offerta.head(2)

Numero di righe del file: 118
cp2021 univoci: 54


,percorso,facolta,corso_studi_id,corso_studi_desc,cp2021,cp2021_desc
0,LAUREE TRIENNALI,Global Law,L-14,Scienze dei servizi giuridici,2.5.2.2.2,Esperti legali in enti pubblici
1,LAUREE TRIENNALI,Global Law,L-14,Scienze dei servizi giuridici,3.4.5.4.0,Tecnici dei servizi di sicurezza privati e pro...


# Raccordo professioni ISTAT CP2021 vs ISCO-08
Il raccordo è disponibile solo per il livello 3 dei codici ISCO-08

In [5]:
raccordo_istat_isco3 = pd.read_excel("./data/classificazioni/raccordo-ISCO08_III-Cp2021_V.xlsx", dtype=str)
raccordo_istat_isco3.columns = ['occupation3d_id', 'occupation3d', 'cp2021', 'cp2021_desc']
raccordo_istat_isco3['occupation3d_id'] = 'OC' + raccordo_istat_isco3['occupation3d_id']
raccordo_istat_isco3.head(2)

,occupation3d_id,occupation3d,cp2021,cp2021_desc
0,OC011,Commissioned armed forces officers,9.1.1.1.0,Ufficiali delle forze armate
1,OC021,Non-commissioned armed forces officers,9.2.1.1.0,"Sergenti, sovraintendenti e marescialli delle ..."


In [7]:
offerta_isco08_liv3 = offerta.merge(raccordo_istat_isco3[['occupation3d_id', 'occupation3d', 'cp2021']], how='inner', on='cp2021')
offerta_isco08_liv3.to_clipboard()

## Creazione tabella per raccordo manuale a livello 4
- Parto dalla lista univoca delle professioni ISTAT
- Join con la tabella delle classificazioni ISCO-08 a 4 cifre (le righe si sdoppiano)
- L'utente manualmente dovrà scegliere il codice a 4 cifre piiù attinente con la descrizione cp2021 ISTAT

In [5]:
# Tabella ISCO a 4 digit
isco08 = pd.read_excel("./data/classificazioni/ISCO-08 EN Structure and definitions.xlsx", dtype=str)
isco08_lev4 = isco08.query("Level=='4'")[['ISCO 08 Code', 'Title EN', 'Definition', 'Tasks include']]
isco08_lev4.columns = ['occupation4d_id', 'occupation4d', 'occupation4d_definitions', 'occupation4d_tasks']
isco08_lev4['occupation3d_id'] = 'OC'+isco08_lev4['occupation4d_id'].str[0:3]
isco08_lev4['occupation4d_id'] = 'OC'+isco08_lev4['occupation4d_id']

# Lista univoca delle professioni che cerchiamo duplicata per ISCO 4 digit
offerta_isco4 = (offerta_isco08_liv3[['cp2021', 'cp2021_desc', 'occupation3d_id']].drop_duplicates()
                 .merge(isco08_lev4, how='left', on=['occupation3d_id'])
                 .sort_values(by=['cp2021', 'occupation4d_id'])
                )
offerta_isco4.to_excel("./data/classificazioni/raccordo_cp2021_isco4_base.xlsx", index=False)
offerta_isco4.head()

,cp2021,cp2021_desc,occupation3d_id,occupation4d_id,occupation4d,occupation4d_definitions,occupation4d_tasks
138,2.5.1.1.1,Specialisti della gestione nella Pubblica Ammi...,OC242,OC2421,Management and Organization Analysts,Management and organization analysts assist or...,(a) assisting and encouraging the development...
139,2.5.1.1.1,Specialisti della gestione nella Pubblica Ammi...,OC242,OC2422,Policy Administration Professionals,Policy administration professionals develop an...,(a) liaising and consulting with programme ad...
140,2.5.1.1.1,Specialisti della gestione nella Pubblica Ammi...,OC242,OC2423,Personnel and Careers Professionals,Personnel and careers professionals provide pr...,(a) advising on and performing personnel func...
141,2.5.1.1.1,Specialisti della gestione nella Pubblica Ammi...,OC242,OC2424,Training and Staff Development Professionals,Training and staff development professionals p...,Tasks include -\n(a) identifying training nee...
142,2.5.1.1.2,Specialisti del controllo nella Pubblica Ammin...,OC242,OC2421,Management and Organization Analysts,Management and organization analysts assist or...,(a) assisting and encouraging the development...


# Selezione OJA in target
Ciclo per tutti i codici ISCO a 3 digit

Perimetro considerato:
- ITALIA
- dal 3° trimestre 2022 in poi
- Livello di istruzione almeno laurea
- sorgenti dati stabili

In [ ]:
%%time

for index, row in tqdm(offerta_isco08_liv3[['occupation3d_id']].drop_duplicates().iterrows()):
    ocse3 = row['occupation3d_id']

    for anno in [2024, 2023]:

        cursor.execute(f"""
                            SELECT oja_id
                                , first_active_date
                                , first_active_month
                                , first_active_year
                                , last_active_date
                                , language
                                , occupation4d_id
                                , occupation3d_id
                                , skill_id
                                , skill_hier3_id
                                , skill_hier2_id
                                , skill_hier1_id
                                , skill_hier0_id
                                , city_id
                                , nuts3_id
                                , country_id
                                , contract_id
                                , education_id
                                , economic_activity2d_id
                                , economic_activity1d_id
                                , activity_sector_id
                                , salary_id
                                , working_time_id
                                , experience_id
                                , source_id
                                , source_category_id
                            FROM wih_oja_latest.wih_oja_blended
                            WHERE source_stability='stable'
                                AND education_id IN ('ED6', 'ED7', 'ED8')
                                AND country='ITALIA'
                                AND first_active_year={anno}
                                --AND (first_active_year>=2024 OR (first_active_year=2023 AND first_active_month>=10))
                                AND occupation3d_id='{ocse3}';
                        """)
        as_pandas(cursor).to_parquet(f"./data/db_target_it/{ocse3}_{anno}.parquet")

15it [18:11, 74.92s/it] 

In [8]:
offerta_isco08_liv3[['occupation3d_id']].drop_duplicates()

,occupation3d_id
0,OC261
1,OC541
3,OC333
4,OC334
5,OC335
7,OC341
9,OC332
11,OC331
14,OC312
30,OC422


In [14]:
ocse3='OC251'
anno=2023
cursor.execute(f"""
                SELECT oja_id
                                , first_active_date
                                , first_active_month
                                , first_active_year
                                , last_active_date
                                , language
                                , occupation4d_id
                                , occupation3d_id
                                , skill_id
                                , skill_hier3_id
                                , skill_hier2_id
                                , skill_hier1_id
                                , skill_hier0_id
                                , city_id
                                , nuts3_id
                                , country_id
                                , contract_id
                                , education_id
                                , economic_activity2d_id
                                , economic_activity1d_id
                                , activity_sector_id
                                , salary_id
                                , working_time_id
                                , experience_id
                                , source_id
                                , source_category_id
                            FROM wih_oja_latest.wih_oja_blended
                            WHERE source_stability='stable'
                                AND education_id IN ('ED6', 'ED7', 'ED8')
                                AND country='ITALIA'
                                AND first_active_year={anno}
                                --AND (first_active_year>=2024 OR (first_active_year=2023 AND first_active_month>=10))
                                AND occupation3d_id='{ocse3}';
                """)
as_pandas(cursor).to_parquet(f"./data/db_target_it/{ocse3}_{anno}.parquet")

In [17]:
ocse3='OC251'
anno=2023
df = pd.read_parquet(f"./data/db_target_it/{ocse3}_{anno}.parquet")
df

,_col0
0,543870
